In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import classification_report
import seaborn as sn; sn.set(font_scale=1.4)
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tqdm import tqdm
import os
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [2]:
class_names=['buildings', 'forest', 'glacier','mountain','sea','street']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}
nb_classes=len(class_names)
print(class_names_label)
image_size=(150,150)


{'buildings': 0, 'forest': 1, 'glacier': 2, 'mountain': 3, 'sea': 4, 'street': 5}


In [3]:
#load dataset
def load_data():
    directory=os.path.abspath("")
    directory=directory + "\dataset"
    #print(directory)
    category=['seg_train','seg_test']
    output=[]
    for eachcategory in category:
        #print(eachcategory)
        path=os.path.join(directory,eachcategory)
        #print(path)
        images=[]
        labels=[]
        #print('Loading {}'. format(eachcategory) + '...')
        
        for folder in os.listdir(path):
            label=class_names_label[folder]
            for file in os.listdir(os.path.join(path, folder)):
                img_path=os.path.join(os.path.join(path,folder),file)
                #print(img_path)
                image=cv2.imread(img_path)
                image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image=cv2.resize(image,image_size)
                
                images.append(image)
                labels.append(label)
        images=np.array(images, dtype='float32')
        labels=np.array(labels, dtype='int32')
        
        output.append((images,labels))
    return output
print('Data loaded...')

Data loaded...


In [4]:
(train_images, train_labels), (test_images, test_labels) = load_data()

In [22]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=25)
print(len(train_labels))

14034


In [23]:
def display_example(class_name, images, labels):
    figsize=(15,15)
    fig=plt.figure(figsize=figsize)
    #fig.subtitle("Some examples of images from the datset", fontsize=16) - deprecated?
    for i in range(10):
        plt.subplot(5,5, i+1)
        plt.yticks([])
        plt.xticks([])
        plt.grid(False)
        #image=cv2.resize(images[i], figsize)
        #plt.imshow(image.astype(np.uint8))
        plt.imshow(images[i].astype(np.uint8))
        plt.xlabel(class_names[labels[i]],fontsize = 8)
    plt.show()

display_example(class_names,train_images,train_labels)

In [26]:
#model = Sequential()

#model.add(Conv2D(32,(3,3), activation='relu', input_shape=(150, 150,3)))
#model.add(MaxPooling2D((2,2)))
#model.add(Conv2D(32,(3,3), activation='relu'))
#model.add(MaxPooling2D((2,2)))
#model.add(Flatten())
#model.add(Dense(64, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))
#print(train_images[1])
type(train_images[1])

numpy.ndarray

In [8]:
model = Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(6,activation=tf.nn.softmax)
    ])

In [9]:
#model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [10]:
#model.fit(train_images, train_labels, batch_size=64,epochs=4)
history=model.fit(train_images, train_labels, batch_size=128,epochs=4,validation_split=0.2)

Epoch 1/4
88/88 [==============================] - 190s 2s/step - loss: 84.0022 - accuracy: 0.2985 - val_loss: 1.5577 - val_accuracy: 0.3912
Epoch 2/4
88/88 [==============================] - 192s 2s/step - loss: 1.3156 - accuracy: 0.4985 - val_loss: 1.4685 - val_accuracy: 0.4029
Epoch 3/4
88/88 [==============================] - 178s 2s/step - loss: 0.9583 - accuracy: 0.6443 - val_loss: 1.4302 - val_accuracy: 0.4731
Epoch 4/4
88/88 [==============================] - 177s 2s/step - loss: 0.6225 - accuracy: 0.7648 - val_loss: 1.6113 - val_accuracy: 0.4838


In [11]:
def plot_accuracy_loss(history):
    fig=plt.figure(figsize=(20,10))
    plt.subplot(221)
    plt.plot(history.history['accuracy'], 'bo--', label='acc')
    plt.plot(history.history['val_accuracy'], 'ro--', label='val_acc')
    plt.title('train_acc VS val_acc')
    plt.ylabel('accuracy')
    plt.xlabel('epochs')
    plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
    plt.rcParams['axes.facecolor'] = 'white'
    plt.legend(fontsize = "large")

    
    plt.subplot(222)
    plt.plot(history.history['loss'], 'bo--', label='loss')
    plt.plot(history.history['val_loss'], 'ro--', label='val_loss')
    plt.title('train_loss VS val_loss')
    plt.ylabel('loss')
    plt.xlabel('epochs')
    plt.grid(color = 'grey', linestyle = '--', linewidth = 0.5)
    plt.rcParams['axes.facecolor'] = 'white'
    plt.legend()

    
    plt.show()   

In [12]:
plot_accuracy_loss(history)

In [13]:
print("We have " + str(len(train_images)) + " of images in " + str(len(class_names_label)) + " classes")
print("And " + str(len(test_images)) + " test images")

We have 14034 of images in 6 classes
And 3000 test images


In [14]:
pred_images = model.predict(test_images) #this is vector of probabilities
pred_labels = np.argmax(pred_images, axis=1) #take the highest one prob.
print(classification_report(test_labels, pred_labels))

94/94 [==============================] - 15s 157ms/step
              precision    recall  f1-score   support

           0       0.39      0.32      0.35       437
           1       0.81      0.68      0.74       474
           2       0.39      0.54      0.45       553
           3       0.45      0.41      0.43       525
           4       0.32      0.29      0.30       510
           5       0.55      0.56      0.55       501

    accuracy                           0.47      3000
   macro avg       0.48      0.47      0.47      3000
weighted avg       0.48      0.47      0.47      3000



In [15]:
#Precision is the ratio of the correctly +ve labeled by our program to all +ve labeled
#Precision answers the following: How many of those who we labeled as diabetic are actually diabetic
#Recall is the ratio of the correctly +ve labeled by our program to all who are diabetic in reality.
#Recall answers the following question: Of all the people who are diabetic, how many of those we correctly predict?
#F1 Score considers both precision and recall.
#It is the harmonic mean(average) of the precision and recall

In [31]:
directory=os.path.abspath("")
test_image_filename='testimage1.jpg'
img_path=os.path.join(directory,test_image_filename)

image=cv2.imread(img_path)
image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image=cv2.resize(image,image_size)

figsize=(5,5)
fig=plt.figure(figsize=figsize)
plt.imshow(image)
plt.grid(False)
plt.show
image.reshape

y_pred=model.predict(image.reshape(1, 150,150,3))

pred_label = np.argmax(y_pred, axis=1)
print("Results of image classification:", y_pred)
np.set_printoptions(suppress=True)
print("Rounded predict values:", np.around(y_pred, decimals=2))
print("Our image belongs to class:", class_names[pred_label[0]])

1/1 [==============================] - 0s 56ms/step
Results of image classification: [[0.9368854  0.00001877 0.00879223 0.00099713 0.00000024 0.05330627]]
Rounded predict values: [[0.94 0.   0.01 0.   0.   0.05]]
Our image belongs to class: buildings
